In [12]:
# 환경설정
from dotenv import load_dotenv
load_dotenv()

True

### [1] 요약봇 (Agent)

- 회의록 요약 기능과 이메일 요약 기능을 함수로 작성 > Tools로 생성
    - 서로 다른 프롬프트로 LLM 질의 > 응답 반환
    - `langchain.tools` 패키지의 `Tool` 활용
- agent를 만들어 요청을 처리할 수 있도록 할 것

In [2]:
test_meeting_note = """
주식회사 앨빈 - '슈퍼밴드' 10주년 기념 컴백 회의

2025년 3월 11일 (화) 14:00 - 16:00, 회사 회의실 A

참석자: 김앨빈 대표, 박다람 매니저, 이소리 홍보팀장, 최빛나 마케팅팀장, 강현우 프로듀서, 슈퍼밴드 멤버 전원

회의 시작. 김앨빈 대표 "이제 10주년인데 뭔가 크게 가야 하지 않겠어?" 다들 동의

일정 조율
- 발매일 6월 15일 확정. 멤버들 일정 문제 없음
- 티저 5월 1일부터 하나씩 공개. SNS 활용
- 뮤비 촬영 4월 10~12일. 로케이션 검토 중
- 쇼케이스 6월 14일로 가자. 장소는 아직 미정, 빠른 섭외 필요
- 월드투어? 일단 논의만, 일정 잡긴 어려움

앨범 컨셉 논의
- 앨범명 아이디어: Reignite, Revival, Firestorm (Reignite로 가는 분위기)
- 컨셉: 10년을 돌아보고 다시 불태우는 느낌. 레트로+현대 락 스타일로 가자
- 타이틀곡 가제: 불꽃처럼. 강렬한 락 사운드, 희망적인 가사
- 수록곡 10곡 예정. 기존 곡 리메이크도 넣을지 고민

마케팅 전략
- SNS 티저 영상, 멤버 개인 브이로그 촬영 진행
- 틱톡 챌린지 같은 거 해야 하나? 다들 긍정적 반응
- 팬미팅, 사인회 6월 중에. 세부 일정은 조율 필요
- 미디어 인터뷰 잡아야 함. 음악방송도 적극 참여
- 패션 브랜드 콜라보 제안 있음. 검토 필요

기타 논의
- 굿즈 제작? 팬클럽 한정 프리오더 진행 고려
- 다큐멘터리 제작할까? 가능성 있음, 제작팀 컨택 예정
- 멤버들 건강 스케줄 조정 신경 써야 함

결론
- 3월 말까지 세부 일정 확정
- 다음 회의 3월 25일 14:00

회의 종료 16:00
"""

test_email = """
제목: [마케팅팀 회의 공지] '슈퍼밴드' 10주년 컴백 마케팅 전략 논의  

팀원 여러분,  

안녕하세요. 최빛나 팀장입니다.  
오늘 진행된 '슈퍼밴드' 10주년 컴백 회의 내용을 공유드립니다.  

컴백 일정이 6월 15일로 확정되었고, 이에 맞춰 마케팅 전략을 구체화해야 합니다. 주요 논의 사항은 다음과 같습니다.  

- **SNS 캠페인**: 티저 영상 및 멤버별 브이로그 콘텐츠 기획  
- **틱톡 챌린지**: 팬 참여 유도 방안 논의  
- **오프라인 이벤트**: 팬미팅, 사인회 일정 및 진행 방식 검토  
- **미디어 홍보**: 음악방송 및 인터뷰 스케줄 조율  
- **콜라보레이션**: 패션 브랜드 협업 가능성 검토  

이에 따라 **마케팅팀 내부 회의**를 진행하려 합니다.  
📅 일정: 3월 13일(수) 오후 2시  
📍 장소: 마케팅팀 회의실  

각자 아이디어를 정리해 오시면 원활한 논의가 가능할 것 같습니다.  
질문이나 제안이 있으면 미리 공유해주세요.  

감사합니다.  
최빛나 드림
"""

In [3]:
from langchain_openai import ChatOpenAI

# LLM 생성
model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.2
)

# 회의록 요약 기능 (함수)
def summary_note_tool(text):
    prompt = f'''
    다음 조건에 맞게 회의 내용을 요약해줘.
    
    <조건>
    1. 회의 장소, 일자, 참여자는 각각 작성할 것
    2. 회의 내용을 대주제, 소주제로 나누어 작성할 것
    3. 요약한 회의 내용은 불렛 형식으로 작성할 것

    <요약할 회의 내용>
    {text}
'''
    return model.invoke(prompt)

# 이메일 요약 기능 (함수)
def summary_email_tool(text):
    prompt = f'''
    다음 조건에 맞게 이메일 내용을 요약해줘.

    <조건>
    1. 이메일 발신자를 따로 표시할 것
    2. 메일 내용을 간략하게 요약해서 한 줄로 표현할 것
    3. 요청받은 업무 사항이 있으면 불렛 형식으로 표현할 것

    <요약할 이메일 내용>
    {text}
'''
    return model.invoke(prompt)

In [ ]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

# Tool 생성 > Agent 생성 > 테스트 데이터로 확인
tools = [
    Tool(
        name='회의 요약',
        func=summary_note_tool,
        description='긴 회의록 내용을 요약한다.'
    ),
    Tool(
        name='이메일 요약',
        func=summary_email_tool,
        description='업무 이메일 내용을 요약한다.'
    )
]

agent = initialize_agent(
    tools,
    model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_error=True,
    verbose=True
)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_2636\332404190.py:18: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [5]:
result = agent.run("회의 내용 요약: " + test_meeting_note)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_2636\3016655772.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run("회의 내용 요약: " + test_meeting_note)




> Entering new AgentExecutor chain...
I need to summarize the meeting content regarding the 10th anniversary comeback of 'Superband' held by Alvin Corporation.

Action: 회의 요약  
Action Input: 주식회사 앨빈 - '슈퍼밴드' 10주년 기념 컴백 회의 내용  
Observation: content="- **회의 장소**: 주식회사 앨빈 회의실  \n- **일자**: 2023년 10월 15일  \n- **참여자**: 앨빈 경영진, 마케팅 팀, 아티스트 매니지먼트 팀, 프로모션 팀  \n\n### 대주제: '슈퍼밴드' 10주년 기념 컴백 계획\n\n- **소주제 1: 컴백 일정 및 전략**\n  - 10주년 기념 앨범 발매 일정 논의\n  - 컴백 전 티저 영상 및 포스터 공개 계획\n  - 팬들과의 소통을 위한 온라인 이벤트 일정 설정\n\n- **소주제 2: 프로모션 및 마케팅**\n  - SNS 및 유튜브를 통한 홍보 전략 수립\n  - 팬미팅 및 라이브 공연 계획\n  - 협업 아티스트 및 브랜드와의 파트너십 논의\n\n- **소주제 3: 콘텐츠 제작**\n  - 뮤직비디오 촬영 장소 및 콘셉트 결정\n  - 앨범 수록곡 리스트 및 작곡가 선정\n  - 팬 참여형 콘텐츠 아이디어 브레인스토밍\n\n- **소주제 4: 예산 및 자원 배분**\n  - 컴백 관련 예산안 검토 및 승인\n  - 각 팀별 자원 배분 및 역할 분담 논의\n\n- **소주제 5: 향후 일정 및 후속 조치**\n  - 다음 회의 일정 설정\n  - 각 팀별 진행 상황 점검 및 피드백 계획 마련" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 348, 'prompt_tokens': 121, 'total_tokens': 469,

In [6]:
print(result)

**회의 요약**  
- **회의 장소**: 주식회사 앨빈 회의실  
- **일자**: 2025년 3월 11일  
- **참여자**: 김앨빈 대표, 박다람 매니저, 이소리 홍보팀장, 최빛나 마케팅팀장, 강현우 프로듀서, 슈퍼밴드 멤버 전원  

### 대주제: '슈퍼밴드' 10주년 기념 컴백 계획  

- **소주제 1: 컴백 일정**  
  - 발매일: 6월 15일  
  - 티저 공개: 5월 1일부터  
  - 뮤비 촬영: 4월 10~12일  
  - 쇼케이스: 6월 14일  

- **소주제 2: 앨범 컨셉**  
  - 앨범명: Reignite  
  - 컨셉: 10년을 돌아보고 다시 불태우는 느낌, 레트로+현대 락 스타일  
  - 타이틀곡 가제: 불꽃처럼  

- **소주제 3: 마케팅 전략**  
  - SNS 티저 영상 및 멤버 개인 브이로그 촬영  
  - 팬미팅 및 사인회: 6월 중  
  - 미디어 인터뷰 및 음악방송 참여  

- **소주제 4: 기타 논의**  
  - 굿즈 제작 및 팬클럽 한정 프리오더  
  - 다큐멘터리 제작 가능성 검토  

- **결론**  
  - 3월 말까지 세부 일정 확정  
  - 다음 회의: 3월 25일 14:00


In [7]:
result = agent.run("메일 내용 요약: " + test_email)



> Entering new AgentExecutor chain...
I need to summarize the email content regarding the marketing team's meeting about the 'Superband' 10th anniversary comeback strategy.

Action: 이메이 요약  
Action Input: 제목: [마케팅팀 회의 공지] '슈퍼밴드' 10주년 컴백 마케팅 전략 논의  

팀원 여러분,  

안녕하세요. 최빛나 팀장입니다.  
오늘 진행된 '슈퍼밴드' 10주년 컴백 회의 내용을 공유드립니다.  

컴백 일정이 6월 15일로 확정되었고, 이에 맞춰 마케팅 전략을 구체화해야 합니다. 주요 논의 사항은 다음과 같습니다.  

- **SNS 캠페인**: 티저 영상 및 멤버별 브이로그 콘텐츠 기획  
- **틱톡 챌린지**: 팬 참여 유도 방안 논의  
- **오프라인 이벤트**: 팬미팅, 사인회 일정 및 진행 방식 검토  
- **미디어 홍보**: 음악방송 및 인터뷰 스케줄 조율  
- **콜라보레이션**: 패션 브랜드 협업 가능성 검토  

이에 따라 **마케팅팀 내부 회의**를 진행하려 합니다.  
📅 일정: 3월 13일(수) 오후 2시  
📍 장소: 마케팅팀 회의실  

각자 아이디어를 정리해 오시면 원활한 논의가 가능할 것 같습니다.  
질문이나 제안이 있으면 미리 공유해주세요.  

감사합니다.  
최빛나 드림

Observation: content="**발신자:** 최빛나 팀장\n\n**요약:** '슈퍼밴드' 10주년 컴백 마케팅 전략 회의 및 내부 회의 일정 안내.\n\n**요청 사항:**\n- 아이디어 정리 후 내부 회의 참석\n- 질문이나 제안 미리 공유" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 393, 'total_

In [8]:
print(result)

**발신자:** 최빛나 팀장  
**요약:** '슈퍼밴드' 10주년 컴백 마케팅 전략 회의 및 내부 회의 일정 안내.  
**요청 사항:**  
- 아이디어 정리 후 내부 회의 참석  
- 질문이나 제안 미리 공유


### [2] News 매니저

1. 최신 뉴스 검색 (상위 3개의 뉴스만 사용)
    - `langchain.utilities`패키지의 `SerpAPIWrapper`
2. 뉴스 요약 및 키워드 분석
3. 사용자 맞춤형 뉴스 추천

In [9]:
!pip install google-search-results

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32173 sha256=b678475f1a899e5fc0d5180b1e0e341a599a3a4643f7d533d5526d4552c93cb4
  Stored in directory: c:\users\playdata\appdata\local\pip\cache\wheels\0c\47\f5\89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


  DEPRECATION: Building 'google-search-results' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'google-search-results'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [18]:
from langchain.llms import OpenAI

# LLM 설정
llm = OpenAI(temperature=0, max_tokens=256)

In [13]:
from langchain.utilities import SerpAPIWrapper


# 최신 뉴스 검색 함수 (상위 3개의 뉴스만 사용)
search_api = SerpAPIWrapper(
    params={'tbm': 'nws', 'num': 3}
)

def search_news(query):
    search_results = search_api.run(query)

    if isinstance(search_results, list):
        search_results = search_results[:3]

    return search_results

In [19]:
# 뉴스 요약 및 키워드 분석 함수
def summary_news(news_text):
    shortened_text = news_text[:500]
    prompt = f'다음 뉴스 내용을 간략하게 요약하고 핵심 키워드를 분석하세요.:\n\n{shortened_text}'
    return llm.invoke(prompt)

In [20]:
# 사용자 맞춤형 뉴스 추천 함수
def recommend_news(topic):
    prompt = f'{topic}에 대한 최신 뉴스를 검색하고, 사용자에게 유용한 정보를 추천하세요.'
    return llm.invoke(prompt)

In [21]:
# Tools & Agent 생성
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

search_tool = Tool(name='웹 검색', func=search_news, description='사용자가 요청한 정보를 검색하고, 상위 3개의 결과를 제공합니다.')
summary_tool = Tool(name='뉴스 요약', func=summary_news,description='검색된 뉴스 내용을 요약하고 키워드를 분석합니다.')
rcmd_tool = Tool(name='맞춤형 뉴스 추천', func=recommend_news, description='사용자 관심사에 맞는 뉴스를 추천합니다.')

tools = [search_tool, summary_tool, rcmd_tool]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [22]:
# 수행
search_result = agent.run('최신 AI 기술 뉴스 하나만 보여줘')
search_result



> Entering new AgentExecutor chain...
 I should use the tools available to find the latest AI technology news.
Action: 맞춤형 뉴스 추천
Action Input: AI 기술
Observation: 

AI 기술에 대한 최신 뉴스를 검색하여 사용자에게 유용한 정보를 추천하는 프로그램을 만들기 위해서는 다음과 같은 단계를 거쳐야 합니다.

1. 데이터 수집
AI 기술에 대한 최신 뉴스를 검색하기 위해서는 먼저 데이터를 수집해야 합니다. 이를 위해 다양한 뉴스 사이트나 블로그, 포럼 등에서 AI 기술에 관련된 기사나 글을 크롤링하여 데이터를 수집합니다. 또한, 소셜 미디어에서도 AI 기술에 대한 최신 소식을 수집할 수 있습니다.

2. 데이터 전처리
수집한 데이터는 다양한 형태와 포맷으로 저장되어 있을 수 있으므로, 이를 통일된 형태로 전처리해야 합니다
Thought: I should clean and organize the collected data to make it easier to analyze.
Action: 데이터 전처리
Action Input: 수집한 AI 기술 뉴스 데이터
Observation: 데이터 전처리 is not a valid tool, try one of [웹 검색, 뉴스 요약, 맞춤형 뉴스 추천].
Thought: I should use the data to find the latest AI technology news.
Action: 웹 검색
Action Input: AI 기술 뉴스
Observation: [{'position': 1, 'link': 'https://www.cio.com/article/4051146/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%99%80-%EC%8B%A4%EB%AC%B4-%ED%98%84%EC%9E%A5%EC%97%90%EC%84%9C-%ED%99%95%EC%9D%B8%ED%95%9C-ai-%EB%8

'AI 기술 뉴스 요약을 통해 최신 AI 기술 뉴스를 요약하고, 맞춤형 뉴스 추천을 통해 사용자에게 유용한 정보를 제공할 수 있습니다.'

In [24]:
search_result = agent.run('요즘 유행하는 건강 식품에 대한 최신 뉴스 알려줘.')
search_result



> Entering new AgentExecutor chain...
 I should use the 웹 검색 tool to find news articles about trending health foods.
Action: 웹 검색
Action Input: "유행하는 건강 식품 뉴스"
Observation: [{'position': 1, 'link': 'https://news.nate.com/view/20250821n02899', 'title': '"비식품 카테고리가 뜬다" 편의점 하반기 트렌드는 \'이것\'', 'source': '네이트', 'date': '2 weeks ago', 'published_at': '2025-08-20 22:01:00 UTC', 'snippet': '한눈에 보는 오늘 : 경제 - 뉴스 : 지난 19일 CU가 개최한 2025 하반기 상품 컨벤션 현장.cCU 편의점 업계가 올 하반기 비식품 카테고리 확장,...', 'favicon': 'https://serpapi.com/searches/68ba414fa3ce84b9c9502fab/images/ce180b19959f245b69d70ac2f65d421de0981a729237f22f.png', 'thumbnail': 'https://serpapi.com/searches/68ba414fa3ce84b9c9502fab/images/ce180b19959f245ba1cc2e215d43f698288e5fc2ac68f4fec2c994424d1b7334.jpeg'}, {'position': 2, 'link': 'https://www.thinkfood.co.kr/news/articleView.html?idxno=102315', 'title': '[2025 건기식 트렌드] 건강기능식품, 이제 모두의 시장…맞춤형 웰니스 지향 확산', 'source': '식품음료신문', 'date': 'Mar 24, 2025', 'published_at': '2025-03-24 07:00:00 UTC', 'snippet'

'최근 건강 식품에 대한 관심이 높아지면서 다양한 뉴스가 나오고 있습니다. 이 중에서도 사용자에게 유용한 정보를 추천해 드리겠습니다. 건강 식품의 종류와 효능에 대한 기사를 추천해 드립니다. 예를 들어, 녹용이 면역력 강화에 좋다는 기사나 콜라겐이 피부 건강에 좋다는 기사 등을 추천해 드립니다. 또한 사용자의 관심사에 따라 맞춤형 뉴스를 추천해 드리기 때문에 사용자가 원하는 정보를 쉽게 얻을 수 있습니다.'